# SST数据的EEMD分解
得到全球平均的SST时间序列之后，我们先不直接拿来做订正，因为原始时间序列长度是百年尺度的非线性非平稳序列，本身会包含多种时间尺度（如季节、年、十年、几十年等）的信息，直接订正效果未必好。如果将原始时间序列分解成多种时间尺度的序列，就相当于得到多种更为丰富的时间序列信息，同时有了不同时间尺度的物理约束。

基于上述的想法，我们采用[EEMD（Ensemble Empirical Mode Decomposition）](https://blog.csdn.net/liu_xiao_cheng/article/details/83897034)方法对原始时间序列来做，EEMD能将原始数据分解成多个从高频到低频的序列，也称本征模函数IMF（Intrinsic Mode Function），分解后的IMF加和也能以可忽略的误差范围接近原始序列。我们可以按照不同时间尺度对IMF组合，将各个组合时间序列输入到机器学习模型里做订正，最后将订正后的组合加起来得到最终的订正时间序列，这样比原始序列直接订正会更精确，且有较清楚的物理约束。

所以，这一章，我们的目的很明确，就是定义EEMD函数，然后调用函数去分解观测和模式数据。

**大家要特别注意这个notebook下使用的环境。 我们要使用的镜像为octave 测试镜像-song-v1，由于python3中的eemd分解时间序列暂时存在残差较大的问题，所以这里Kernel类型我们选择Octave（语法基本与Matlab一致），使用前人已经建立成熟的eemd分解代码。因为在Octave下运行eemd函数的速度普遍比matlab慢得多，集合数稍一增加，计算用时将会变长很多（大概2-3分钟），所以计算资源这里大家可以选择2核8G CPU资源，如希望算的快推荐大家选择4核CPU资源。**


**这里严格说还是属于海表面温度数据处理的部分，但是由于环境不同，所以我们单独运行。**

## 3.1 读取数据


In [ ]:
Path_Data = './dataset'
% 读取结构体数组，这里用到fullfile函数，作用是利用文件各部分信息创建并合成完整文件名。
SST_ERSST_AreaMean = load(fullfile(Path_Data, 'SST_ERSST_AreaMean.mat') ); % 185401-201412
SST_Historical_AreaMean = load( fullfile(Path_Data, 'SST_Historical_AreaMean.mat') );% 185401-201412
SST_SSP126_AreaMean = load( fullfile(Path_Data, 'SST_SSP126_AreaMean.mat') );% 201501-210012
SST_SSP245_AreaMean = load( fullfile(Path_Data, 'SST_SSP245_AreaMean.mat') );% 201501-210012
SST_SSP585_AreaMean = load( fullfile(Path_Data, 'SST_SSP585_AreaMean.mat') );% 201501-210012
% 读取结构体中的数据，包括全球平均SST的观测数据、模式历史数据和未来预估数据
SST_ERSST_AreaMean = SST_ERSST_AreaMean.SST_ERSST_AreaMean;
SST_Historical_AreaMean = SST_Historical_AreaMean.SST_Historical_AreaMean;
SST_SSP126_AreaMean = SST_SSP126_AreaMean.SST_SSP126_AreaMean;
SST_SSP245_AreaMean = SST_SSP245_AreaMean.SST_SSP245_AreaMean;
SST_SSP585_AreaMean = SST_SSP585_AreaMean.SST_SSP585_AreaMean;
% 数据分解选取的时间长度选择86年，与未来情景的时间长度相同，也能保证历史时期数据和未来情景预估数据在分解后的时间分量数目相同。
Num_Year = 86; % 192901-201412
Num_Sample_SST = Num_Year*12; 
Index_192901 = (1929-1854)*12+1 % 1929年1月在数组的位置
SST_ERSST_AreaMean = SST_ERSST_AreaMean(Index_192901:end); % 取出时间192901-201412的数据
SST_Historical_AreaMean = SST_Historical_AreaMean( Index_192901:end); % 192901-201412

## 3.2 定义EEMD函数
这里我们需要定义两个用于EEMD数据分解的函数，一个是求极值的函数”extrema“，另一个是主函数—“eemd”函数。
大家在调用函数的时候只需要输入两个参数和待分解的变量，里面的细节感兴趣可以自行学习。

### 3.2.1 定义extrema函数
extrema函数定义好后在主函数eemd中被调用。

**注：第一行的%%file extrema.m 千万不要动
**

In [ ]:
%%file extrema.m
function [spmax, spmin, flag]= extrema(in_data)
    % 该子函数可以找到序列的极大值及其位置，极小值及其位置，然后用三次样条函数来形成包络。
    % 函数：function [spmax, spmin, flag]= extrema(in_data)
    % 输入:
    %       in_data: 输入数据, 被研究的时间序列。
    % 输出：
    %       spmax: 极大值的位置（第1列）及其对应的值（第2列）。
    %       spmin: 极大值的位置（第1列）及其对应的值（第2列）。
    %
    % 注意事项:
    %       EMD使用三次样条曲线作为数据的最大和最小包络，除寻找样条曲线外，还要注意端点。
    %
    % 参考：
    % 代码编者：吴召华 
    % 标注：S.C.Su
    % 翻译：匡志远
    % 代码中有两段独立的循环。
    % part1: 找到最大值及其位置，处理起止点。
    % part2: 找到最小值及其位置，处理起止点。
    %
    % 在eemd中调用该函数。
 
    flag=1;
    dsize=length(in_data);
    %part1: 找到最大值及其位置，处理起止点。
    % 起点
    spmax(1,1) = 1;
    spmax(1,2) = in_data(1);
    jj=2;
    kk=2;
    while jj<dsize,
        if ( in_data(jj-1)<=in_data(jj) && in_data(jj)>=in_data(jj+1) )  % && is in octave; & is in matlab
            spmax(kk,1) = jj;
            spmax(kk,2) = in_data (jj);
            kk = kk+1;
        end
        jj=jj+1;
    end
    %终点
    spmax(kk,1)=dsize;
    spmax(kk,2)=in_data(dsize);
    
    if kk>=4
        slope1=(spmax(2,2)-spmax(3,2))/(spmax(2,1)-spmax(3,1));
        tmp1=slope1*(spmax(1,1)-spmax(2,1))+spmax(2,2);
        if tmp1>spmax(1,2)
            spmax(1,2)=tmp1;
        end
 
        slope2=(spmax(kk-1,2)-spmax(kk-2,2))/(spmax(kk-1,1)-spmax(kk-2,1));
        tmp2=slope2*(spmax(kk,1)-spmax(kk-1,1))+spmax(kk-1,2);
        if tmp2>spmax(kk,2)
            spmax(kk,2)=tmp2;
        end
    else
        flag=-1;
    end
 
    msize=size(in_data);
    dsize=max(msize);
    xsize=dsize/3;
    xsize2=2*xsize;
 
    %part2: 找到最小值及其位置，处理起止点。.
    spmin(1,1) = 1;
    spmin(1,2) = in_data(1);
    jj=2;
    kk=2;
    while jj<dsize,
        if ( in_data(jj-1)>=in_data(jj) && in_data(jj)<=in_data(jj+1))  % && is in octave; & is in matlab.
            spmin(kk,1) = jj;
            spmin(kk,2) = in_data (jj);
            kk = kk+1;
        end
        jj=jj+1;
    end
    spmin(kk,1)=dsize;
    spmin(kk,2)=in_data(dsize);
 
    if kk>=4
        slope1=(spmin(2,2)-spmin(3,2))/(spmin(2,1)-spmin(3,1));
        tmp1=slope1*(spmin(1,1)-spmin(2,1))+spmin(2,2);
        if tmp1<spmin(1,2)
            spmin(1,2)=tmp1;
        end
 
        slope2=(spmin(kk-1,2)-spmin(kk-2,2))/(spmin(kk-1,1)-spmin(kk-2,1));
        tmp2=slope2*(spmin(kk,1)-spmin(kk-1,1))+spmin(kk-1,2);
        if tmp2<spmin(kk,2)
            spmin(kk,2)=tmp2;
        end
    else
        flag=-1;
    end
 
    flag=1;
 
end

### 3.2.2 定义eemd函数
定义完extrema函数之后，接下来我们定义主函数eemd，后面我们对SST数据分解，只需要调用这个函数。

In [ ]:
%%file eemd.m
function allmode=eemd(Y,Nstd,NE)
    % 该函数用于EMD/EEMD
    % 函数：function allmode=eemd(Y,Nstd,NE)
    % 输入:
    %       Y:输入数据，一维
    %       Nstd: 加入噪声的标准差与输入数据标准差的比值，一般在0-0.2之间。
    %       NE: EEMD使用的集合数，常取50，100，200等值。
    % 输出：
    %       输出 N*(m+1)的数组, N是输入数据Y的长度，m=fix(log2(N))-1，
    %       第一列是原始数据，第2列，3列，。。。，m列是从高频到低频的IMF, 第(m+1)列是残差项，也是趋势项。
    %
    % 注意事项:
    %       Nstd = 0,NE = 1时，EEMD变为EMD。
    %       筛选次数为10次，停止准者不能更改。
    %
    % 参考： Wu, Z., and N. E Huang (2008), Ensemble Empirical Mode Decomposition: a noise-assisted data analysis method. 
    %       Advances in Adaptive Data Analysis. Vol.1, No.1. 1-41.
    % 代码编者：吴召华 
    % 标注：S.C.Su
    % 翻译：匡志远
    % 代码中有三个循环。
    % 1.读取数据，找出标准差std,输入数据除std
    % 2.设TNM为IMF的总数，TNM2为IMF总数与原始数据、剩余残差的条目，即TNM2 = TNM + 2，将0分配给TNM2数组
    %【3.-------------------------------------循环1：EEMD，进行NE次循环开始-------------------------------------
    %     4.添加噪声
    %     5.筛选前给定初始值
    %    【6.---------------------------循环2：开始计算IMF,寻找IMF循环开始---------------------------
    %         【7.------------------循环3：筛选10次得到IMF,循环开始及结束------------------】
    %          8.经过10次筛选，得到IMF
    %          9.从原始数据中减去IMF,对剩余部分循环找到下一个IMF
    %     6.---------------------------在得到所有IMF之后，循环2结束---------------------------】
    %     9.得到所有IMF,最后得到残差，即趋势项
    % 3.-------------------------------------NE次EEMD分解结果求和完毕，循环1结束-------------------------------------】
    % 10.求和结果减去除NE,再乘上标准差。
    %
    % 该函数常用于1维时间序列的EEMD分解，且停止准则固定。

    %part1.读取数据，找出标准差std,输入数据除std
    xsize=length(Y);
    dd=1:1:xsize;
    Ystd=std(Y);
    Y=Y/Ystd;

    %part2.设TNM为IMF的总数，TNM2为IMF总数与原始数据、剩余残差的条目，即TNM2 = TNM + 2，将0分配给TNM2数组
    TNM=fix(log2(xsize))-1;
    TNM2=TNM+2;
    for kk=1:1:TNM2, 
        for ii=1:1:xsize,
            allmode(ii,kk)=0.0;
        end
    end

    %part3.循环1：EEMD，进行NE次循环开始
    for iii=1:1:NE
        if mod(iii,10)== 0
             disp(['Now ensemble number is : ', num2str(iii)])

        end
        %part4. 添加噪声
        for i=1:xsize
            temp=randn(1,1)*Nstd;
            X1(i)=Y(i)+temp;
        end

        %part4.把原始值放在第一列  
        for jj=1:1:xsize
            mode(jj,1) = Y(jj);
        end
    
        %part5.筛选前给定初始值
        xorigin = X1;
        xend = xorigin;
        
        %part6.循环2：开始计算IMF,寻找IMF循环开始
        nmode = 1;
        while nmode <= TNM
            xstart = xend; 
                        
            iter = 1;      
    
            %part7.循环3：筛选10次得到IMF,循环开始
            while iter<=10
                [spmax, spmin, flag]=extrema(xstart);  %调用子函数extrema 
                %用spline函数拟合包络
                upper= spline(spmax(:,1),spmax(:,2),dd); %上界 
                lower= spline(spmin(:,1),spmin(:,2),dd); %下界
                mean_ul = (upper + lower)/2; %上界和下界的平均值 
                xstart = xstart - mean_ul; % 从上一次初始值减掉平均值
                iter = iter +1;
            end
            %part7.循环3：筛选10次得到IMF,循环结束   
            
            %part8.经过10次筛选，得到IMF,
            xend = xend - xstart;
            nmode=nmode+1;
            
            %part9. 从原始数据中减去IMF,对剩余部分循环找到下一个IMF,储存IMF
            for jj=1:1:xsize
                mode(jj,nmode) = xstart(jj);
            end

        end
        %part6.在得到所有IMF之后，循环2结束

        %part 10.得到所有IMF之后就能得到残差项，将残差项放在mode的最后一列
        for jj=1:1:xsize
            mode(jj,nmode+1)=xend(jj);
        end
        % allmode 包含3部分：original；IMF; trend 
        allmode=allmode+mode;
        
    end
    %part3.NE次EEMD分解结果求和完毕，循环1结束

    %part10.求和结果减去除NE,再乘上标准差。
    allmode=allmode/NE;
    allmode=allmode*Ystd;

end



## 3.3 EEMD分解观测和模式SST
定义好分解的函数，下一步我们就可以分解数据了。

**大家要特别注意这里一个地方：调用eemd函数时前面不要加注释，但可以在后面加注释。这应该是在同一个notebook里面定义函数并调用时容易出错的点。**

eemd分解除了输入原始时间序列，还需要输入两个参数，一个是加入噪声的标准差与输入数据标准差的比值，也就是Nstd，一般在0-0.2之间取值。另一个是 EEMD使用的集合数，也就是NE，常取50，100，200等值。
这里，对观测和模式数据我们均设置相同的值，即Nstd取0.2，NE取100。


### 3.3.1 分解观测SST

In [ ]:
% 设置两个参数
Nstd = 0.2;
NE = 100;
% 开始对观测数据分解，并计时
tic
EEMD_ERSST = eemd( SST_ERSST_AreaMean, Nstd, NE ); % 得到第一列是原始序列，第二列到最后一列是分解的IMF分量
toc  
IMFs_ERSST = EEMD_ERSST(:, 2:end)';
NIMFs_ERSST= size(IMFs_ERSST,1);
Recons_IMFs_ERSST = sum(IMFs_ERSST, 1);
Delta_IMFs_ERSST = SST_ERSST_AreaMean - Recons_IMFs_ERSST;
min(Delta_IMFs_ERSST0), max(Delta_IMFs_ERSST) % 可以看看残差的最大值最小值
% 画图，可以先不画，后面我们把数据保存好了再下一章python3环境下画。
% t = 1:Num_Sample_SST;
% hf = figure;
% for n = 1:NIMFs_ERSST
%   subplot(NIMFs_ERSST, 1, n);
%    plot(t, IMFs_ERSST(n,:) );
%    ylabel( ['IMF', num2str(n) ] );
% end 
% xlabel( 'Time' );

### 3.3.2 分解模式的历史SST
分解过程与观测数据类似，只需调用eemd函数。

In [ ]:
Nstd = 0.2;
NE = 100;
tic
EEMD_Historical = eemd( SST_Historical_AreaMean, Nstd, NE );
toc
IMFs_Historical = EEMD_Historical(:, 2:end)';
NIMFs_Historical= size(IMFs_Historical,1);
Recons_IMFs_Historical = sum(IMFs_Historical, 1);
Delta_IMFs_Historical = SST_Historical_AreaMean - Recons_IMFs_Historical;


### 3.3.3 分解模式的未来预估SST
同样地，分解过程与观测数据类似，只需调用eemd函数。

**SSP126**

In [ ]:
Nstd = 0.2;
NE = 100;
tic
EEMD_SSP126 = eemd( SST_SSP126_AreaMean, Nstd, NE );
toc
IMFs_SSP126 = EEMD_SSP126(:, 2:end)';
NIMFs_SSP126= size(IMFs_SSP126,1);
Recons_IMFs_SSP126 = sum(IMFs_SSP126, 1);
Delta_IMFs_SSP126 = SST_SSP126_AreaMean - Recons_IMFs_SSP126;


**SSP245**

In [ ]:
Nstd = 0.2;
NE = 100;
tic
EEMD_SSP245 = eemd( SST_SSP245_AreaMean, Nstd, NE );
toc
IMFs_SSP245 = EEMD_SSP245(:, 2:end)';
NIMFs_SSP245= size(IMFs_SSP245,1);
Recons_IMFs_SSP245 = sum(IMFs_SSP245, 1);
Delta_IMFs_SSP245 = SST_SSP245_AreaMean - Recons_IMFs_SSP245;


**SSP585**

In [ ]:
Nstd = 0.2;
NE = 100;
tic
EEMD_SSP585 = eemd( SST_SSP585_AreaMean, Nstd, NE );
toc
IMFs_SSP585 = EEMD_SSP585(:, 2:end)';
NIMFs_SSP585= size(IMFs_SSP585,1);
Recons_IMFs_SSP585 = sum(IMFs_SSP585, 1);
Delta_IMFs_SSP585 = SST_SSP585_AreaMean - Recons_IMFs_SSP585;


## 3.4 保存数据
保存eemd分解的数据，留作后续机器学习订正使用。

In [ ]:
% 设置一下保存路径
Path_Pre = './SaveData/'
Path_SaveData =  [Path_Pre , 'EEMD/']
if ~exist(Path_SaveData)
    mkdir(Path_SaveData)
    disp( [Path_SaveData, ' successfully build!'] )
else
    disp( [Path_SaveData, ' has been existed!'] )
end
% 保存变量
save( fullfile(Path_SaveData, 'IMFs_ERSST.mat'), 'IMFs_ERSST',  '-v7' ) 
save( fullfile(Path_SaveData, 'IMFs_Historical.mat'), 'IMFs_Historical',  '-v7' )
save( fullfile(Path_SaveData, 'IMFs_SSP126.mat'), 'IMFs_SSP126',  '-v7' )
save( fullfile(Path_SaveData, 'IMFs_SSP245.mat'), 'IMFs_SSP245',  '-v7' )
save( fullfile(Path_SaveData, 'IMFs_SSP585.mat'), 'IMFs_SSP585',  '-v7' )

## 小结
本章我们带领大家从头定义了用于做数据分解的eemd函数，然后对观测和模式SST数据做了分解，最后将数据保存。下一章，我们将基于分解后的数据做一个绘图展示，然后计算各个分量的时间尺度，再根据我们关心的时间尺度对各个分量进行组合，得到我们最终要输入机器学习模型中的订正时间序列对象。我们主要是想带领大家在python环境下运行代码，所以仅仅用Octave完成SST的eemd分解这一部分。
